In [121]:
from LinearAES import AES
from cryptolib import KEY, ascii_letters
from os import urandom
aes = AES(KEY)


In [122]:
[chr(d) for d in aes._transpose([c for c in b"0123456789abcdef"])]

['0',
 '4',
 '8',
 'c',
 '1',
 '5',
 '9',
 'd',
 '2',
 '6',
 'a',
 'e',
 '3',
 '7',
 'b',
 'f']

In [132]:
import numpy as np
# Function to get the n-th bit of a byte (0-indexed from the right)
def get_bit(value, n):
    return (value >> n) & 1

# Function to evaluate the linear approximation for an S-box
def linear_approximation(i):
    # Initialize a table to count the approximations
    count_table = np.zeros((8, 8), dtype=int)
    
    # Check all input-output pairs for bias
    for input_val in range(256):  # Input value (0 to 255)
        output_val = aes.encrypt(b"crypto{"+bytes([input_val])+urandom(8))[i]  # S-box output
        
        # For each input and output bit, check for linear approximations
        for input_bit_pos in range(8):  # Check all input bits
            for output_bit_pos in range(8):  # Check all output bits
                # aes of input and output bits
                input_bit = get_bit(input_val, input_bit_pos)
                output_bit = get_bit(output_val, output_bit_pos)
                
                # Track how often input bit equals output bit
                if input_bit == output_bit:
                    count_table[input_bit_pos][output_bit_pos] += 1
    
    return count_table

# Analyze the linear approximations for the S-box
count_table = [linear_approximation(i) for i in range(16)]
for i in range(100):
    for j in range(len(count_table)):
        count_table[j] += linear_approximation(j)

# Print the bias table
print("Linear Approximation Bias Table (counts of equal input and output bits):")
print(count_table)

# Convert the counts to probabilities and biases (how far from 128, since random should be ~128 times)
bias_table = [np.abs(ct - 128) / 256.0 for ct in count_table]

print("\nBias Table (deviation from random 0.5):")
print(bias_table)

Linear Approximation Bias Table (counts of equal input and output bits):
[array([[12796, 12865, 12844, 13014, 12881, 12997, 12963, 12905],
       [13092, 12931, 13048, 12816, 12835, 12939, 13011, 13009],
       [12902, 12975, 12910, 13068, 13099, 12945, 12871, 12985],
       [12918, 12971, 12858, 12930, 13045, 12869, 12957, 12853],
       [12978, 12985, 12890, 12870, 12875, 12911, 12953, 12913],
       [12670, 12919, 12778, 12908, 13097, 12955, 12959, 12969],
       [12876, 12963, 12884, 12990, 13133, 12909, 13035, 12955],
       [12980, 12957, 12886, 12940, 12985, 12869, 12985, 12997]]), array([[12977, 12979, 12891, 12956, 12913, 12935, 12872, 12942],
       [13027, 12937, 12873, 12892, 12783, 12887, 12834, 12954],
       [12951, 12897, 12973, 12814, 12977, 13087, 12950, 12938],
       [12935, 12879, 12857, 13006, 12743, 13019, 12930, 12864],
       [12939, 12901, 12785, 12836, 12947, 12847, 12840, 12818],
       [12985, 12847, 12871, 13010, 12963, 13011, 12942, 12974],
       [12889,

In [119]:
from os import urandom
temp = 0
for i in range(0,256):
    initial = get_bit(i, 0)
    final = get_bit(aes.encrypt(b"\x00"*0+bytes([i])+b"\x00"*15)[0], 4)
    temp+=initial==final
print(temp)

256


In [58]:
np.bincount(np.array([get_bit(aes.encrypt(b"\x00"*0+bytes([i])+b"\x00"*15)[0], 6) for i in range(0,256)]))

array([128, 128])